# 목적
    - 데이터가 바이너리(엑셀) 형식으로 제공되면 파일형태로 다운로드 후 수집 처리 해야 되는 case
    - 석유공사에서 제공하는 실시간 주유가격 사이트에서 지역별 가격 데티터 획득
    - 사이트는 시도 선택 -> 시군구 선택 -> 엑셀 저장 버튼 클릭후 다운로드된다.
        - 엑셀을 다 모으면 파이썬에서 모든 파일을 읽어서 하나의 df로 구성 (그래야 db로 간다) -> 저장
        - 모든 데이터 저장이 완료되면 엑셀 파일을 이동 혹은 삭제
    - ajax로 각 구간 통신이 진행된다 -> LEVEL 4 수준의 selenium 사용
    - 빈도 주기는 1일 1회 진행
        - 변경시간에 맞춰서 주기를 맞추면 된다.

In [6]:
import pandas as pd
import sys
from selenium import webdriver as wd
import random
import time
# step 1 -> 데이터를 수집하는 사이트까지 웹드라이버 가동 후 페이지 접속하시오.
# 초기에 서브 페이지 접속을 거부하는 사이트도 있다 -> 시간 텀을 두고 연속 접속 시도
driver = wd.Chrome('./chromedriver.exe')
target_site = 'https://www.opinet.co.kr/searRgSelect.do'
driver.get(target_site)
time.sleep(2)
driver.get(target_site)

In [9]:
# step 2 -> 시도 select 태그를 찾아서 그밑에 시도를 표현하는 option태그에 value값들을 추출한다.
# #SIDO_NM0 > option 특정

# 데이터추출을 셀레니윰으로 한다.

sidos = driver.find_elements_by_css_selector('#SIDO_NM0 > option')  # 스푸에선 이런식 -> soup.select('.quickSearchResultBoxSidoGugun > li')
# 프럼프트1개(시/도) + 실제데이터 17개
len(sidos)
for sido in sidos:
    # 태그에서 속성값 추출할때 사용 : 셀레니늄에서는 get_attribute('속성값') BeautifulSoup에서는 .get('속성값')
    name = sido.get_attribute('value').strip()
    if name: # 이름있으면
        print(name)


# 내포로 짜기 sidos(걍 위에랑 상관없이 새로운 sidos) = [
#                           sido.get_attribute('value').strip()
#                           for sido in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
#                                    ]


서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


In [11]:
sidos  = [ sido.get_attribute('value').strip()
            for sido in driver.find_elements_by_css_selector('#SIDO_NM0 > option')
            if sido.get_attribute('value').strip()
         ]
sidos

['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [12]:
# 해당 시도를 선택하는 이벤트를 강제로 발생
# 사람이 select 태그를 선택하면 select 태그의 value가 변경          <---- 여기까지 하면 밑에는 통신이 자동됨 그래서 시간두고 기다리기
#  -> 이벤트가 감지해서 -> ajax -> 시군구 전달 -> 화면변경 : 5초대기
# 1. select 태그 특정
select_sido = driver.find_element_by_id('SIDO_NM0')
# 2. 값 변경
select_sido.send_keys(sidos[8])  # -> 홈피 띄어둔 상태서 이거 돌리면 값이 경기도로 변경된다.


In [16]:
# step3 -> 특정 시도 선택, send_key() => 입력 계열의 모든 태그들에게 값을 설정할때 사용
# 
# 전국 17개 시도를 돌면서 값변경(시도값) 진행 5초간격으로

for sido in sidos:
    # 화면 조작시 그 대상이되면 메모리 주소가 변경된다 -> 알 수 없음으로 -> 매번 안전하게 구함
    select_sido = driver.find_element_by_id('SIDO_NM0')
    select_sido.send_keys(sido)
    time.sleep(5)
    print (sido)

서울특별시
부산광역시
대구광역시
인천광역시
광주광역시
대전광역시
울산광역시
세종특별자치시
경기도
강원도
충청북도
충청남도
전라북도
전라남도
경상북도
경상남도
제주특별자치도


In [27]:
# step 4 -> 시군구값들중 하나 선택 -> 잠시후 -> 엑셀저장 클릭
# 1차로 서울시의 자치구 이름(값)을 추출한다.
sigungus = driver.find_elements_by_css_selector('#SIGUNGU_NM0 > option')

gus = list()
for sigungu in sigungus:
    gu = sigungu.get_attribute('value')
    if gu:
        gus.append(gu)

len(gus)

25

In [28]:
# 자치구 이름을 하나씩 추출

for gu in gus:    
    print( gu )
    # 해당 구 바로 선택 -> 값변경 -> 화면껌벅(form 전송됨) -> 해당구의 주유소 정보 세팅된다 :대기
    driver.find_element_by_id('SIGUNGU_NM0').send_keys( gu )
    time.sleep(10)
    # 클릭(엑셀 저장 버튼)
    driver.find_element_by_id('glopopd_excel').click()
    #break

강남구
강동구
강북구
강서구
관악구
광진구
구로구
금천구
노원구
도봉구
동대문구
동작구
마포구
서대문구
서초구
성동구
성북구
송파구
양천구
영등포구
용산구
은평구
종로구
중구
중랑구


In [29]:
# step 5 -> 엑셀들을 다 모아서 읽어서 df로 구성
# 특정 폴더에 특정 형식의 파일들을 다 모아라(경로+ 이름(파일명))
import glob

files = glob.glob('C://Users//USER//Downloads//*.xls')
len(files), files[0]

(25, 'C://Users//USER//Downloads\\지역_위치별(주유소) (1).xls')

In [46]:
# xls -> DataFrame으로 끌어올리기
import pandas as pd

# 상위 row 데이터 2개는 skip, 3번째 row 칼럼, 4번째부터 데이터 ->pd.read_excel(files[0], header=2)

# df들을 담는 그릇
tmp = list()
for file in files:  #파일스에서 파일 하나씩 꺼내시오
    # 파일 읽어서 -> df생성 -> 그릇에 담는다
    tmp.append(pd.read_excel(file, header=2))
len(tmp) , print(tmp)

# df 25개를 생성 하여 1개의 df로 병합

[       지역                    상호                            주소      상표  \
0   서울특별시  재건에너지 재정제2주유소 고속셀프지점  서울특별시 강동구  천호대로 1246 (둔촌제2동)  현대오일뱅크   
1   서울특별시     현대오일뱅크㈜직영 명일셀프주유소          서울 강동구 고덕로 168 (명일동)  현대오일뱅크   
2   서울특별시                구천면주유소         서울 강동구 구천면로 357 (암사동)  현대오일뱅크   
3   서울특별시        (주)삼표에너지 고덕주유소           서울 강동구 고덕로 39 (암사동)   GS칼텍스   
4   서울특별시          대성석유(주)길동주유소              서울 강동구 천호대로 1168   GS칼텍스   
5   서울특별시        (주)소모에너지 신월주유소        서울 강동구 양재대로 1323 (성내동)   GS칼텍스   
6   서울특별시        지에스칼텍스㈜ 동서울주유소        서울 강동구 천호대로 1456 (상일동)   GS칼텍스   
7   서울특별시               방아다리주유소          서울 강동구 동남로 811 (명일동)   SK에너지   
8   서울특별시              sk해뜨는주유소         서울 강동구  상일로 132 (강일동)   SK에너지   
9   서울특별시       (주)퍼스트오일 코알라주유소     서울특별시 강동구  올림픽로 556 (성내동)   S-OIL   
10  서울특별시      주)지유에너지직영 오렌지주유소          서울 강동구 성안로 102 (성내동)   SK에너지   
11  서울특별시        (주)소모에너지 성내주유소         서울 강동구 올림픽로 578 (성내동)   GS칼텍스   
12  서울특별시               천호현대주유소           서울 강동구 천

(25, None)

In [45]:
#df를 하나로 합치기
full_df = pd.concat(tmp)
full_df.shape
# 서울시 주유소는 총 443개 존재한다, 10개의 속성(컬럼, 특징)을 표현한것

(443, 10)